In [ ]:
import pandas as pd
import matplotlib.pyplot as plt



In [ ]:
# Load S&P 500 index data from CSV
sp500_df = pd.read_csv('S&P500.csv', parse_dates=['Date'])
sp500_df = sp500_df.set_index('Date')



In [ ]:
# Load individual stock data from CSV
# For example, let's say you want to compare Apple (AAPL) with S&P 500
apple_df = pd.read_csv('AAPL.csv', parse_dates=['Date'])
apple_df = apple_df.set_index('Date')



In [ ]:
# Select the 'Close' price column for both dataframes
sp500_close = sp500_df['Close']
apple_close = apple_df['Close']

In [ ]:
# Plot the S&P 500 and Apple stock prices
plt.figure(figsize=(10, 5))
plt.plot(sp500_close, label='S&P 500')
plt.plot(apple_close, label='Apple (AAPL)')
plt.legend()
plt.title('S&P 500 vs. Apple Stock Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

In [ ]:
# Animate the plot
ani = FuncAnimation(fig, update, frames=len(sp500_df), blit=True, repeat=False)

plt.title('S&P 500 vs. Apple Stock Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

In [ ]:
# Importing module 
import mysql.connector
 
# Creating connection object
mydb = mysql.connector.connect(
    host = "localhost",
    user = "yourusername",
    password = "your_password"
)
 
# Printing the connection object 
print(mydb)